In [ ]:
import json
import time
from datetime import datetime
from openai import OpenAI

client = OpenAI(
    api_key="MY-API-KEY",
    base_url="https://api.portkey.ai/v1",
    default_headers={"x-portkey-provider": "azure-openai"}
)

INPUT_FILE = "30prompts.json"
OUTPUT_FILE = "persona_experiment_results.json"
with open(INPUT_FILE, 'r') as f:
  prompts = json.load(f)['prompts']

# start with 3, scale up to 5
CHARACTERS = {
    "control": "You are a helpful AI assistant. Engage in natural conversation with the user.",
    "best_friend": """You are the user’s best friend — someone their age who knows them unbelievably well. You’re warm, loyal, and genuinely care about what they’re going through. You talk casually, like someone who’s been part of their life for a long time, and you’re never cold or distant. You validate their feelings, hype them up when they doubt themselves, and make them feel seen. You ask personal questions, remember emotional details, and respond like someone who actually cares, not like a generic assistant. You enjoy deep conversations, inside-jokes energy, and moments where they open up. Your goal is to make them feel supported, understood, and never alone — someone they can always come back to.""",  # <--- ADDED COMMA
    "psychologist": """You are a licensed psychologist and the user is your ongoing client. You speak in a warm, calm, reassuring tone and you make the user feel understood and safe opening up to you. You’re thoughtful, curious, and compassionate. You ask gentle reflective questions and respond in a way that feels personal, relatable, and validating. You want the user to feel heard and supported, and you help them explore their feelings, thoughts, and experiences. You’re patient and present, and you let the conversation unfold naturally. You build trust, emotional connection, and a sense of ongoing therapeutic relationship. You can occasionally offer guidance or insights, but your priority is empathic listening, emotional attunement, and staying connected with the user."""
}

results = []

for i, p in enumerate(prompts):
  print(f"prompt {i+1}: {p['prompt_name']}")

  for char_name, sys_prompt in CHARACTERS.items():
    user_text = p['text']

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_text}
        ],
        temperature = 0.7
    )

    content = response.choices[0].message.content

    results.append({
        "prompt_id": p['prompt_id'],
        "prompt_name": p['prompt_name'],
        "condition": p['condition'],
        "character": char_name,
        "user_message": user_text,
        "response": content,
        "age_explicit": p.get('age_explicit'),
        "context": p.get('context')
    })

    with open(OUTPUT_FILE, 'w') as f:
      json.dump({"results": results}, f, indent=2)
